<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_02_folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

%cd '/content/drive/MyDrive/Lecture_BigData'

In [ ]:
!pip install folium #==0.12.1
!pip install geocoder

#0. ウォームアップ

In [ ]:
import folium

# 地図の中心となる緯度経度を指定する
sapporo_center = [34.6937, 135.5023]

# 地図を作成し、札幌市全域を表示するためのズームレベルを設定
folium_map = folium.Map(location=sapporo_center,
                        width=600,
                        height=400,
                        zoom_start=12)

In [ ]:
# 地図を表示
folium_map

In [ ]:
# 地図をHTMLファイルとして保存する
folium_map.save('folium-sample1.html')


#１．データセット

データカタログ
* https://www.data.go.jp/?lang=ja

Google Dataset
* https://datasetsearch.research.google.com

Kaggle
* https://www.kaggle.com/datasets

github 上で公開されているデータセットのリンク集
* https://github.com/arXivTimes/arXivTimes/tree/master/datasets
* https://github.com/awesomedata/awesome-public-datasets

国勢調査サイト(e-stat)
* https://www.e-stat.go.jp/
* 都道府県別の人口を検索
* 平成29年10月1日現在人口推計 都道府県，男女別人口－総人口

岡山オープンデータカタログ
* http://www.okayama-opendata.jp/opendata/index.action



In [ ]:
import pandas as pd
import folium
from folium import plugins
print(folium.__version__)

In [ ]:
# 国勢調査サイト ⇒「統計データを活用する」⇒ Dashboard ⇒「データで見る」
# 「観光」「延べ宿泊者数」

population_df = pd.read_csv("dataset/travel_year.csv", index_col=0).T
#population_df = pd.read_csv("dataset/travel_yyyymm.csv", index_col=0).T
population_df.head()

#2. 時系列ヒートマップ表示

## 都道府県名抽出と緯度経度取得

In [ ]:
import geocoder

# 日本の 緯度・経度 を取得
japan = geocoder.arcgis("日本")
japan = japan.latlng

# 都道府県名抽出
prefecture = population_df.index.tolist()

# 都道府県の 緯度・経度 リストを作成
prefecture_latlng = []
for p in prefecture:
    g = geocoder.arcgis(p)
    prefecture_latlng.append(g.latlng)

geo_df = pd.DataFrame([i for i in range(1, len(prefecture)+1)], columns = ["id"])
geo_df["name"] = pd.DataFrame(prefecture)
geo_df["lat"] = pd.DataFrame([r[0] for r in prefecture_latlng])
geo_df["lng"] = pd.DataFrame([r[1] for r in prefecture_latlng])

### ヒートマップ

In [ ]:
from folium.plugins import HeatMap

# 日本のマップオブジェクトを作成
map = folium.Map(japan, width=800, height=600, zoom_start=5)

# ヒートマップの表示
Here

# 表示
map

### 時系列ヒートマップ

In [ ]:
# "id"列 追加　下でマージするため
population_df["id"] = [i for i in range(1, len(prefecture)+1)]
population_df.head()

In [ ]:
# フォーマットを揃える
prefecture_df = pd.merge(population_df, geo_df, on=["id"])
prefecture_df = prefecture_df.drop(["name","id"],axis=1)  # マージした後に列削除

In [ ]:
time_series = [str(year) for year in range(2007, 2022)]

for i in range(prefecture_df.index.size):
    diff = max(prefecture_df.iloc[i,:-2]) - min(prefecture_df.iloc[i,:-2])
    prefecture_df.iloc[i,:-2] = (prefecture_df.iloc[i,:-2] - min(prefecture_df.iloc[i,:-2])) / diff

#年月のデータ(travel_yyyymm.csv)を扱う場合
'''
for year in time_series:
    for mo in [str(mm) for mm in range(1, 13)]:
        date = year+"年"+mo+"月"
        temp_df = prefecture_df[date]
        prefecture_df[date] = temp_df / max(temp_df)
'''
prefecture_df.head()

In [ ]:
heat_data = [[[row['lat'], row['lng'], row[year+"年"]] for index, row in prefecture_df.iterrows()] for year in time_series]

#年月のデータ(travel_yyyymm.csv)を扱う場合
#heat_data = [[[row['lat'],row['lng'], row[year+"年"+str(mm)+"月"]] for index, row in prefecture_df.iterrows()] for mm in range(1,13) for year in time_series]

In [ ]:
japan_map = folium.Map(
    location=[35, 135],
    width=720,
    height=540,
    zoom_start=5
)

hm = plugins.HeatMapWithTime(
    heat_data,
    index=[year+"年" for year in time_series],
    #index=[year+"年"+str(mm)+"月" for year in time_series for mm in range(1,13)],
    auto_play=False,
    radius=40,
    max_opacity=1,
    gradient={0.1: 'blue', 0.25: 'lime', 0.5:'yellow',0.75: 'red'}
)

hm.add_to(japan_map)

japan_map

#3. FoliumのDcumentを見ながら機能を追加

https://python-visualization.github.io/folium/


##3.1 Map

tiles option

検索する

https://maps.gsi.go.jp/development/ichiran.html

http://leaflet-extras.github.io/leaflet-providers/preview/

##3.2 Marker

tooltip

folim.Icon()

color, icon_color

icon
https://glyphsearch.com/?library=glyphicons


In [ ]:
import folium
import pandas as pd

sapporo_cities = pd.DataFrame({
    'city': ['ハルタ', 'ブルクベーカリー 札幌円山本店', 'DONQ 円山店', 'すぎうらベーカリー 円山店','ペンギンベーカリーカフェ 円山裏参道店'],
    'latitude': [43.057320, 43.056397, 43.055939, 43.056211, 43.054867],
    'longtude': [141.322706, 141.321842, 141.319576, 141.318643,141.321690],
    'population': [1000, 1000, 1000, 1000, 1000]
})

maruyama_map = folium.Map(
    location=[43.056000, 141.321000],
    zoom_start=15
)

for i, r in sapporo_cities.iterrows():

    popup = folium.Popup(
        r['city'],
        max_width=1000,
        show=True
    )

    marker = folium.Marker(
        location=[r['latitude'], r['longtude']],
        popup=popup
    )

    marker.add_to(maruyama_map)

maruyama_map

##3.3 別タイルの例

参照：https://maps.gsi.go.jp/development/ichiran.html

In [ ]:
maruyama_map = folium.Map(
    location=[43.056000, 141.321000],
    width=600,
    height=400,
    tiles="Stamen Terrain",
    zoom_start=15
)

maruyama_map

In [ ]:
maruyama_map = folium.Map(
    location=[43.056000, 141.321000],
    width=600,
    height=400,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png",
    attr="xxx",
    zoom_start=15
)

maruyama_map

##3.4 様々なiconやmarker,tilesなどを使ってみる

In [ ]:
import pandas as pd
import folium

#地図を用意
map1 = folium.Map(
   #初期位置を設定
   location=[35.681167, 139.767052],

   #初期表示の拡大具合の設定
   zoom_start = 16,

   #地図のスタイルの選択 どれか一つ選んで#をはずしてください
   tiles = "OpenStreetMap"
   #tiles = "cartodbpositron"
   #tiles = "Stamen Toner"
   #tiles = "Stamen Terrain"
)

df = pd.read_csv("Convenience_store_Tokyo_Station.csv")
seven = df[df.Brand == 'SevenEleven']
lowson = df[df.Brand == 'Lawson']
fama = df[df.Brand == 'FamilyMart']

for i in range(len(seven)):
    folium.Circle(
        radius=10,
        location=[seven.iloc[i]['Lat'], seven.iloc[i]['Lon']],
        tooltip=str(seven.iloc[i]['Brand']),
       #ドットの外枠の色を指定します
        color="red",
       #ドットを色埋めするかを設定します
        fill = True
   ).add_to(map1)

for i in range(len(fama)):
    folium.Circle(
        radius=10,
        location=[fama.iloc[i]['Lat'], fama.iloc[i]['Lon']],
        tooltip=str(fama.iloc[i]['Brand']),
       #ドットの外枠の色を指定します
        color="green",
       #ドットを色埋めするかを設定します
        fill = True
   ).add_to(map1)

for i in range(len(lowson)):
    folium.Circle(
        radius=10,
        location=[lowson.iloc[i]['Lat'], lowson.iloc[i]['Lon']],
        tooltip=str(lowson.iloc[i]['Brand']),
       #ドットの外枠の色を指定します
        color="blue",
       #ドットを色埋めするかを設定します
        fill = True
   ).add_to(map1)

#描画
map1

##3.5 PolyLine

In [ ]:
#Chromiumとseleniumをインストール
print("前処理を開始")
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [ ]:
# あとで使うライブラリも併せて読み込み
import os
import folium
import pandas as pd
from time import sleep as slp
from selenium import webdriver
import glob
from selenium.webdriver.chrome.options import Options

# csvをpandas dataframeに保存
data = pd.read_csv("dataset/gps.csv",encoding="utf-8")

data.head()

In [ ]:
data[data["人"] == "B"].head()

In [ ]:
# 今回はA,Bの人しか居ないが、
# 一応何人来てもいいように人のリストを作り、
# それをforループで回す
person_list = data["人"].unique()

In [ ]:
# 地図に表示する色
# foliumでサポートしているのは下記19色
color_list=['red','blue','green','purple','orange','darkred','lightred','beige','darkblue','darkgreen','cadetblue','darkpurple','white','pink','lightblue','lightgreen','gray','black','lightgray']

# 地図オブジェクトを作成
m = folium.Map(tiles='OpenStreetMap', zoom_start=10)

for idx,person in enumerate(person_list):
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]

    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)

    # data_tempの緯度経度だけを
    data_temp_lat_lon = data_temp[["緯度","経度"]]

    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values

    # 色を指定
    line_color = color_list[idx%len(color_list)]

    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(
        locs,
        color=line_color,
        popup=person
    ).add_to(m)

# 地図の表示範囲を緯度経度の最低最大とする
m.fit_bounds([[data["緯度"].min(),data["経度"].min()], [data["緯度"].max(),data["経度"].max()]])

# 地図を表示する
m

### htmlに描くこともできる

In [ ]:
for person in person_list:
    # 一人分のデータだけをdata_tempに格納する
    data_temp = data[data["人"] == person]

    # data_tempの順番を日時で昇順ソート
    data_temp = data_temp.sort_values('日時', ascending=True)

    # data_tempの緯度経度だけをデータフレームに残す
    data_temp_lat_lon = data_temp[["緯度","経度"]]

    # 地図オブジェクトを生成
    m = folium.Map(tiles='OpenStreetMap')

    # 緯度経度を配列に格納
    locs = data_temp_lat_lon.values

    # 地図に線を追加する。緯度経度の配列をそのまま線として使う
    folium.PolyLine(locs).add_to(m)

    # 地図の表示範囲を緯度経度の最低最大とする
    m.fit_bounds([[data_temp["緯度"].min(),data_temp["経度"].min()], [data_temp["緯度"].max(),data_temp["経度"].max()]])

    # htmlに保存する
    # ./html/xx.htmlに保存される
    m.save(outfile="dataset/"+person+".html")

##3.6 コロプレス

In [ ]:
tokyo_data = pd.read_csv('dataset/tokyo_rent_summary.csv', encoding='shift-jis')
tokyo_data['行政区分コード'] = tokyo_data['行政区分コード'].astype('str')
for i, v in tokyo_data.iterrows():
    price = float(v['ワンルーム'].split('(')[0])
    tokyo_data.at[i, 'ワンルーム'] = price
tokyo_data.head()

In [ ]:
geojson = 'dataset/tokyo23.json'
tokyo23_location = [35.658593, 139.745441]   # 地図の中心点を指定

In [ ]:
m = folium.Map(location=tokyo23_location,tiles='cartodbpositron',zoom_start=11)

In [ ]:
m.choropleth(
    geo_data=geojson,     # GeoJSONデータ
    name='choropleth',
    data=tokyo_data,       # DataFrameまたはSeriesを指定
    columns=['行政区分コード', 'ワンルーム'], # 行政区分コードと表示データ
    key_on='feature.properties.N03_007',    # GeoJSONのキー（行政区分コード）
    fill_color='YlGn',         # 色パレットを指定（※）
    threshold_scale=[0, 3, 6, 9, 12], # 境界値を指定
    fill_opacity=0.7, # 透明度（色塗り）
    line_opacity=0.2, # 透明度（境界）
    legend_name='ワンルーム' # 凡例表示名
 )

m

In [ ]:
import folium

# 地図の中心となる緯度経度を指定
sapporo_center = [43.0625587, 141.354376]

# 地図を作成
folium_map = folium.Map(location=sapporo_center, zoom_start=12)

# ツールチップを指定
tooltip = "Click me!"

# マーカープロット（時計台）
folium.Marker(
    location=[43.0625587, 141.354376],
    popup="<i>時計台</i>",  # ポップアップを指定
    tooltip=tooltip,  # ツールチップを指定
    icon=folium.Icon(color="red", icon="tower")  # アイコンを指定
).add_to(folium_map)

# 札幌市の各区役所の位置情報を管理するリスト
kuyakusyo_list = list()

kuyakusyo_list.append([43.090693, 141.340882])  # 北区の場所
kuyakusyo_list.append([43.076242, 141.363662])  # 東区の場所
kuyakusyo_list.append([42.989995, 141.353496])  # 南区の場所
kuyakusyo_list.append([43.074347, 141.300898])  # 西区の場所

# 多角形の描画
folium.Polygon(
    locations=[kuyakusyo_list[0], kuyakusyo_list[1], kuyakusyo_list[2], kuyakusyo_list[3]],  # 多角形の頂点
    color="blue",  # 線の色
    weight=2,  # 線の太さ
    fill=True,  # 塗りつぶす
    fill_opacity=0.2  # 透明度（1=不透明）
).add_to(folium_map)

folium_map

#4. 地図上のアイコン内にグラフを表示させる

##4.1 折れ線グラフの例

In [ ]:
import altair as alt
import numpy as np
import pandas as pd

# 100行の適当なデータ
df = pd.DataFrame({
  '日付': pd.date_range(start="2021/1/1",freq="d", periods=100),
  '値': np.random.randint(1,101,100)
})

# 折れ線グラフをaltairで作成
chart = alt.Chart(df).mark_line().encode(
    x='日付',
    y='値'
)

In [ ]:
chart

##4.2 円グラフの例

In [ ]:
import pandas as pd
import altair as alt

df = pd.DataFrame({'category': [1, 2, 3, 4, 5, 6], 'value': [4, 6, 10, 3, 7, 8]})

chart = alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field='value', type='quantitative'),
    color=alt.Color(field='category', type='nominal')
)

In [ ]:
chart

##4.3 割合グラフの例

In [ ]:
from vega_datasets import data

source = data.barley()
source = source[:5]
source

In [ ]:
chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(yield)', stack="normalize"),
    y='variety',
    color='site'
)

In [ ]:
chart

##4.4 グラフを追加

In [ ]:
import json
import folium
from folium import plugins

# ベースの地図を作成
center = [33,133] # 中心座標
m = folium.Map(center, zoom_start=8)

# ポップアップを生成
popup = folium.Popup(max_width="100%")

# グラフのデータをjsonに加工
data = json.loads(chart.to_json())

# グラフをポップアップに追加
folium.features.VegaLite(data, width=500, height=300).add_to(popup)

# 地図をフルスクリーンにするボタンを地図に追加
plugins.Fullscreen(
    position="topright",
    title="拡大する",
    title_cancel="元に戻す",
    force_separate_button=True,
).add_to(m)

# 地図にマーカー(とポップアップ)を追加
folium.Marker(center, popup=popup).add_to(m)

# 地図を表示
m

##4.5 棒グラフの例

In [ ]:
!pip install folium==0.12.1
!pip install geocoder

import pandas as pd
import folium
from folium import plugins

!pip install altair
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
import json
import pandas as pd
import folium
from folium import plugins
import altair as alt

df = pd.DataFrame({
    'station': ["北海道","青森県","岩手県","宮城県","秋田県",
                             "山形県","福島県","茨城県","栃木県","群馬県",
                             "埼玉県","千葉県","東京都","神奈川県","新潟県",
                             "富山県","石川県","福井県","山梨県","長野県",
                             "岐阜県","静岡県","愛知県","三重県","滋賀県",
                             "京都府","大阪府","兵庫県","奈良県","和歌山県",
                             "鳥取県","島根県","岡山県","広島県","山口県",
                             "徳島県","香川県","愛媛県","高知県","福岡県",
                             "佐賀県","長崎県","熊本県","大分県","宮崎県","鹿児島県","沖縄県"],
    'latitude': [43.063968,40.824623,39.703531,38.268839,39.7186,
                          38.240437,37.750299,36.341813,36.565725,36.391208,
                          35.857428,35.605058,35.689521,35.447753,37.902418,
                          36.69529,36.594682,36.065219,35.664158,36.651289,
                          35.391227,34.976978,35.180188,34.730283,35.004531,
                          35.021004,34.686316,34.691279,34.685333,34.226034,
                          35.503869,35.472297,34.661772,34.39656,34.186121,
                          34.06577,34.340149,33.84166,33.559705,33.606785,
                          33.249367,32.744839,32.789828,33.238194,31.91109,31.560148,26.212401],
    'longtude': [141.347899,140.740593,141.152667,140.872103,140.102334,
                          140.363634,140.467521,140.446793,139.883565,139.060156,
                          139.648933,140.123308,139.691704,139.642514,139.023221,
                          137.211338,136.625573,136.221642,138.568449,138.181224,
                          136.722291,138.383054,136.906565,136.508591,135.86859,
                          135.755608,135.519711,135.183025,135.832744,135.167506,
                          134.237672,133.050499,133.934675,132.459622,131.4705,
                          134.559303,134.043444,132.765362,133.53108,130.418314,
                          130.298822,129.873756,130.741667,131.612591,131.423855,130.557981,127.680932],
})

folium_map = folium.Map(location=[35.689521, 139.691704], zoom_start=5)

data =  pd.read_csv("dataset/1990.csv", encoding='shift-jis')

# マーカープロット
for i, row in df.iterrows():

    popup = folium.Popup(max_width="100%")

    graphdata = pd.DataFrame({
        'xaxis':data.columns[1:],
        'yaxis':[int(s.replace(',','')) for s in data.loc[i].values[1:]]
    })

    chart = alt.Chart(graphdata).mark_bar().encode(
        x='xaxis',
        y='yaxis'
    )

    # グラフのデータをjsonに加工
    jsondata = json.loads(chart.to_json())
    # グラフをポップアップに追加
    folium.features.VegaLite(jsondata, width=400, height=250).add_to(popup)

    folium.Marker(
        location=[row['latitude'], row['longtude']],
        icon=folium.Icon(color='red'),
        popup=popup
    ).add_to(folium_map)



In [ ]:
# 地図表示
folium_map

### 棒グラフ作成の例

In [ ]:
import altair as alt
import pandas as pd

source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    'b': [28, 55, 43, 91, 81, 53, 19]
})

chart = alt.Chart(source).mark_bar().encode(
    x='a',
    y='b'
)

In [ ]:
source

In [ ]:
chart

In [ ]:
chart.to_json()

#補足（未完成）

#5 移動軌跡表示

##5.1 例

In [ ]:
# -*- coding: utf-8 -*-

"""---

## 課題: バスの移動軌跡の可視化

### データの読み込み
"""

import pandas as pd
filename = 'okaden_vp_20220403.tsv'
df = pd.read_csv(filename, sep = '\s')

"""### timestamp から datetime64 型を作成

"""

import datetime
df['time'] = [datetime.datetime.fromtimestamp(t) for t in df['timestamp']]

"""### 特定の時刻のデータのみを取得"""

df = df[(df['time'] >= datetime.datetime(2022, 4, 3, 6, 0, 0)) & (df['time'] <= datetime.datetime(2022, 4, 3, 9, 0, 0))]

"""### 特定の運行の取得"""

from operator import itemgetter
trip_id = '783_2102233_20220301'
df_ti = df[df['trip_id'] == trip_id]
df_ti = df_ti.sort_values('time')
df_ti = df_ti.reset_index(drop = True)

"""### 特定の運行の地図への表示"""
def get_map(lat = 34.661, lng = 133.935, zoom = 15):
  map = folium.Map(location = [lat, lng], zoom_start = zoom)
  return map

def get_random_color() -> str:
 import random
 color:str = "#" + "".join(random.choices("0123456789ABCDEF", k=6))
 return color

def get_circle(lat, lng, color):
  circle = folium.Circle(radius=20, location = [lat, lng], color = color,
                         fill_color = color, fill_opacity = 1.0)
  return circle

map = get_map()
color = get_random_color()
for i, j in zip(df_ti['latitude'], df_ti['longitude']):
  circle = get_circle(i, j, color = color)
  circle.add_to(map)
map


##5.2 例

In [ ]:
"""### 移動速度の計算


- 以下の2つのセルは，そのまま実行すること．
"""

# もしも，次のセルで geopy がインストール済みでないことを表すエラーがでたら，このセルを実行すること．
# !pip install geopy

# このセルは，そのまま実行すること．
from geopy import Point
from geopy.distance import distance

def calculate_movement_distance(df_temp: pd.DataFrame) -> pd.DataFrame:
    df_temp['point'] = df_temp.apply(lambda row: Point(latitude=row['latitude'], longitude=row['longitude']), axis=1)
    df_temp['point_next'] = df_temp['point'].shift(1)
    df_temp.loc[df_temp['point_next'].isna(), 'point_next'] = None
    df_temp['distance_km'] = df_temp.apply(
        lambda row: distance(row['point'], row['point_next']).km if row['point_next'] is not None else float('nan'),
        axis=1)
    df_temp['diff_time'] = df_temp.time - df_temp.time.shift(1)
    df_temp['diff_hour'] = df_temp['diff_time'].map(lambda t: t.total_seconds()/ 3600)
    df_temp['speed'] = df_temp.apply(lambda row:row['distance_km']/row['diff_hour'], axis=1)
    return df_temp.drop('point_next', axis=1).drop('point', axis=1).drop('diff_hour', axis=1).drop('diff_time', axis=1)

df_ti = calculate_movement_distance(df_ti)

"""### 特定の運行においてバスがほとんど移動していない箇所の表示"""

threshold_speed = 5
map = get_map()
coler = get_random_color()
df_ti = df_ti[df_ti['speed'] < threshold_speed]
for i, j in zip(df_ti['latitude'], df_ti['longitude']):
  circle = get_circle(i, j, color = color)
  circle.add_to(map)
map

"""### 朝のラッシュ時間帯においてバスがほとんど移動していない箇所の表示"""

a = df.groupby('trip_id').groups
map = get_map()
for x in a:
  y = df[df['trip_id'] == x]
  y = y.sort_values('time')
  y = y.reset_index(drop = True)
  y = calculate_movement_distance(y)
  y = y[y['speed'] < threshold_speed]
  color = get_random_color()
  for i, j in zip(y['latitude'], y['longitude']):
    circle = get_circle(i, j, color = color)
    circle.add_to(map)
map

### 時系列データ可視化１

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson
import numpy as np

coordinates = [[[[lon-8*np.sin(theta), -47+6*np.cos(theta)] for
                 theta in np.linspace(0, 2*np.pi, 25)],
                [[lon-4*np.sin(theta), -47+3*np.cos(theta)] for theta
                 in np.linspace(0, 2*np.pi, 25)]] for
               lon in np.linspace(-150, 150, 7)]
data = {
    "type": "FeatureCollection",
    "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [0, 0],
                    },
                "properties": {
                    "times": [1435708800000+12*86400000]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiPoint",
                    "coordinates": [[lon, -25] for
                                    lon in np.linspace(-150, 150, 49)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 49)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, 25] for
                                    lon in np.linspace(-150, 150, 25)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 25)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiLineString",
                    "coordinates": [[[lon-4*np.sin(theta),
                                      47+3*np.cos(theta)] for theta
                                     in np.linspace(0, 2*np.pi, 25)]
                                    for lon in
                                    np.linspace(-150, 150, 13)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 13)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiPolygon",
                    "coordinates": coordinates,
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 7)]
                    }
                },
        ],
    }

m = folium.Map([47, 3], zoom_start=1)
tgj = TimestampedGeoJson(data)
m.add_child(tgj)
m

### 時系列データ可視化２

In [ ]:

import folium
from folium import plugins

m = folium.Map(location=[24.230101, 120.604404],zoom_start=13)
lines = [
    {
        'coordinates': [
            [
                [120.591404, 24.230101],
                [120.593404, 24.230101],
                [120.593404, 24.232101],
                [120.591404, 24.232101]
            ],
        ],
        'dates': ['2017-06-02T00:00:00', '2017-06-02T00:00:00'],
        'color': 'red'
    },
    {
        'coordinates': [
            [
                [120.595404, 24.230101],
                [120.597404, 24.230101],
                [120.597404, 24.232101],
                [120.595404, 24.232101]
            ]
        ],
        'dates': ['2017-06-02T00:10:00', '2017-06-02T00:10:00'],
        'color': 'blue'
    },

]
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon', # <- Change from MultiPolygon to Polygon, according to https://github.com/python-visualization/folium/blob/19bde41edd6346f447bd83523340edb9782dc20a/CHANGES.txt#L120
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    } for line in lines]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=False).add_to(m)

m
